In [ ]:
!pip install -q pandas tqdm 
!pip install -q transformers==4.55.0 # llm requires >=4.46.0
!pip install -q safetensors==0.4.3 # downgrade for torch 2.1.0
!pip install -q bitsandbytes==0.43.2 accelerate==1.9.0 # quantization
!pip install -q peft trl # finetune
!pip install -q datasets

In [ ]:
import os, pandas as pd, json, random
from datasets import Dataset, concatenate_datasets
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

In [ ]:
# 객관식 여부 판단 함수
def is_multiple_choice(question_text):
    """
    객관식 여부를 판단: 2개 이상의 숫자 선택지가 줄 단위로 존재할 경우 객관식으로 간주
    """
    lines = question_text.strip().split("\n")
    option_count = sum(bool(re.match(r"^\s*[1-9][0-9]?\s", line)) for line in lines)
    return option_count >= 2


# 질문과 선택지 분리 함수
def extract_question_and_choices(full_text):
    """
    전체 질문 문자열에서 질문 본문과 선택지 리스트를 분리
    """
    lines = full_text.strip().split("\n")
    q_lines = []
    options = []

    for line in lines:
        if re.match(r"^\s*[1-9][0-9]?\s", line):
            options.append(line.strip())
        else:
            q_lines.append(line.strip())
    
    question = " ".join(q_lines)
    return question, options

# 프롬프트 생성기
def make_prompt_auto(row):
    Question = str(row["Question"]).strip()
    Answer = str(row["Answer"]).split("답변:")[-1].strip()
    if is_multiple_choice(Question):
        question, options = extract_question_and_choices(Question)
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
                )
    else:
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n"
                f"질문: {Question}\n\n"
                "답변:"
                )
    response = 

    return prompt, response

In [ ]:
# 데이터 로드

dfs = []
dfs.append(pd.read_csv("../data/CyberMetric/mcqa.csv", "CyberMetric"))
dfs.append(pd.read_csv("../data/FinShibainu/mcqa.csv", "FinShibainu"))
dfs.append(pd.read_csv("../data/FinShibainu/qa.csv", "FinShibainu"))
dfs.append(pd.read_csv("../data/SecBench/mcqa.csv", "SecBench"))
dfs.append(pd.read_csv("../data/SecBench/qa.csv", "SecBench"))

full = pd.concat(dfs, ignore_index=True)

In [ ]:
full

In [ ]:
records = [make_prompt_auto(q) for _, r in full.iterrows()]
random.shuffle(records)

# 간단 split
n = int(len(records)*0.98)
train_ds = Dataset.from_list(records[:n])
eval_ds  = Dataset.from_list(records[n:])

In [ ]:
train_ds

In [ ]:
eval_ds

In [ ]:
# 모델 선택
models = [
    "gemma-ko-7b", # baseline
    "ax-4.0-light-7b", # skt
    # "polyglot-12.8b",
    # "koalpaca-polyglot-12.8b",
    "midm-2.0-11.5b", # kt
    # "HyperCLOVAX-SEED-Think-14B", # naver
    # "kanana-1.5-15.7b-a3b-instruct", # kakao
    # "exaone-4.0-32b" # lg
]
selected_model = models[0]
model_path = f"/workspace/models/{selected_model}" # 로컬 저장 모델 경로

# 4bit 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16, # NaN 방지
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# # 8bit 설정
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,          # 4bit → 8bit
#     llm_int8_threshold=6.0,     # 기본값 (필요 시 조정)
#     llm_int8_has_fp16_weight=False  # True로 하면 일부 레이어 FP16 유지
# )

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    padding_side="left"
)

# 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto", # GPU 자동 배정
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    attn_implementation="eager",
    # trust_remote_code=True # naver
)

In [ ]:
# QLoRA Setting

base_model = prepare_model_for_kbit_training(base_model)

peft_config = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],  # 모델에 맞게
    task_type="CAUSAL_LM"
)
model = get_peft_model(base_model, peft_config)

In [ ]:
# 4) 텍스트 컬럼 지정 (SFT)
def formatting_func(samples):
    # SFTTrainer는 list[str] 반환을 기대. 여기서 "prompt+response"를 결합해서 지도학습
    texts = []
    for p, r in zip(samples["prompt"], samples["response"]):
        texts.append(p + r)
    return texts

cfg = SFTConfig(
    output_dir=f"/workspace/models/{selected_model}_qlora",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    logging_steps=20,
    save_steps=500,
    eval_strategy="steps",
    eval_steps=500,
    bf16=True,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    gradient_checkpointing=True,
    max_seq_length=2048,
    dataset_num_proc=1,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    formatting_func=formatting_func,  # 데이터 전처리 함수
    args=cfg,
)

trainer.train()
trainer.save_model(f"/workspace/models/{selected_model}_qlora/adapter")  # LoRA 어댑터 저장

In [ ]:
# (선택) 추론 편의 위해 베이스와 병합하여 단일 가중치 생성
try:
    merged = trainer.model.merge_and_unload()
    merged.save_pretrained(f"/workspace/models/{selected_model}_qlora/merged")
    tokenizer.save_pretrained(f"/workspace/models/{selected_model}_qlora/merged")
except Exception as e:
    print("merge skipped:", e)